# Lektion 8

In [ ]:
from sympy import *
init_printing()
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Matrixplots

In [ ]:
i = Symbol('i')
j = Symbol('j')

In [ ]:
def hilbert_element(i,j):
    return 1/(i+j+1)

hilbert_element(i,j)

In [ ]:
hilbert = Matrix(20, 20, hilbert_element)

In [ ]:
#plt.imshow(hilbert)  # TypeError

In [ ]:
plt.imshow(np.array(hilbert).astype(float), interpolation='none')
plt.colorbar();

In [ ]:
plt.imshow(np.log(np.array(hilbert).astype(float)), 
           interpolation='none')
plt.colorbar();

In [ ]:
hilbert = Matrix(60, 60, hilbert_element)

In [ ]:
plt.imshow(np.log(np.array(hilbert).astype(float)), 
           interpolation='none')
plt.colorbar();

In [ ]:
inv_hilbert = hilbert**(-1)

In [ ]:
plt.imshow(np.log(np.array(inv_hilbert).astype(float)), 
           interpolation='none')
plt.colorbar();

Was ist das Problem?

In [ ]:
fig = plt.figure(figsize=(12, 5))
fig.add_subplot(121)
plt.imshow(np.log(abs(np.array(inv_hilbert).astype(float))), 
           interpolation='none')
plt.colorbar()
fig.add_subplot(122)
plt.imshow(np.sign(np.array(inv_hilbert).astype(float)), 
           interpolation='none')
plt.colorbar();

Als Ersatz für 3D-Plot:

In [ ]:
x = Symbol('x')
y = Symbol('y')
f = cos(sqrt(x**2+y**2))
fn = lambdify((x,y), f, 'numpy')

In [ ]:
x = np.linspace(-2*np.pi, 2*np.pi)
y = x
X, Y = np.meshgrid(x, y)
W = fn(X, Y)
plt.imshow(W, origin='lower', cmap=plt.cm.coolwarm)
plt.colorbar();